In [1]:
import os
import pandas as pd
os.chdir("/home/adrianhurtado/testing/madness-test-suite/timings-6-3-22")
from madnessReader import MadnessReader
from madnessReader import ExcitedData
from madnessReader import FrequencyData
from daltonRunner import DaltonRunner
import matplotlib.pyplot as plt
dalton_reader = DaltonRunner()

In [2]:
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ','aug-cc-pV5Z','aug-cc-pV6Z']
d_basis_list=['d-aug-cc-pVDZ','d-aug-cc-pVTZ','d-aug-cc-pVQZ','d-aug-cc-pV5Z','d-aug-cc-pV6Z']

In [3]:
def create_polar_table(mol,xc,basis_list,xx):
    
    ground_dalton, response_dalton = dalton_reader.get_frequency_result(mol, 'hf', 'dipole', basis_list[0])
    freq=response_dalton['frequencies']
    g_data={}
    xx_data=[]
    for i in range(len(freq)):
        xx_data.append({})
    for basis in basis_list:
        ground_dalton, response_dalton = dalton_reader.get_frequency_result(mol, 'hf', 'dipole', basis)
        for i in range(len(freq)):
            xx_data[i][basis]=response_dalton[xx][i]
        g_data[basis]=ground_dalton['totalEnergy']
    g_df=pd.Series(g_data)
    g_df.name='Total HF Energy'
    names=[]
    for f in freq:
        raw_f = r'{}'.format(str(f)) 
        #names.append(r'$$\alpha_{xx}('+raw_f+r')$$')
        names.append('a('+'{:.3f}'.format(f)+')')
    r_dfs=[]
    for i in range(len(freq)):
        r_dfs.append(pd.Series(xx_data[i]))
        r_dfs[i].name=names[i]
    dalton_df=pd.concat([g_df]+r_dfs,axis=1)

    moldata=FrequencyData(mol,'hf','dipole')
    moldata.polar_df[xx]
    mad_data_e={}
    mad_data_r={}
    mad_data_e['Total HF Energy']=moldata.ground_e['e_tot']

    for i in range(len(names)):
        mad_data_r[names[i]]=moldata.polar_df[xx][i]
    
    mad_data_e=pd.Series(mad_data_e)
    mad_data_r=pd.Series(mad_data_r)
    
    mad_data=pd.concat([mad_data_e,mad_data_r],axis=0)
    mad_data.name='MRA'
    return dalton_df.append(mad_data)
    
    return dalton_df

def create_data(mol,basis_list):
    
    res_dict={'xx':'density_residualX','yy':'density_residualY','zz':'density_residualZ'}
    
    
    moldata=FrequencyData(mol,'hf','dipole')
    xx=['xx','yy','zz']
    data=[]
    for x in xx:
        data.append(create_polar_table(mol,'hf',basis_list,x))
    average=(data[0]+data[1]+data[2])/3
    
    diff_data=average-average.loc['MRA']
    diff_data=diff_data.drop(index='MRA')
    
    polar_diff=diff_data.drop('Total HF Energy',axis=1)
    
    residuals=moldata.final_density_residuals()
    
    cleanX=residuals['density_residualX'].reset_index().drop('index',axis=1)
    cleanY=residuals['density_residualY'].reset_index().drop('index',axis=1)
    cleanZ=residuals['density_residualZ'].reset_index().drop('index',axis=1)
    
    cleanX.rename(columns={'density_residualX':'MRA density residual'},inplace=True)
    cleanY.rename(columns={'density_residualY':'MRA density residual'},inplace=True)
    cleanZ.rename(columns={'density_residualZ':'MRA density residual'},inplace=True)
    clean=(cleanX+cleanY+cleanZ)/3
    clean.index=polar_diff.T.index
    
    average=average.append(clean.T)
    average.name='Average Polarizability'
    
    
    energy_diff=diff_data['Total HF Energy']
    return average,diff_data,energy_diff,polar_diff



def create_polar_diff_plot(mol,basis_list):
    
    
    title=mol
    yl=r' $\Delta\alpha_{avg}$'+r' (MRA - BASIS)'
    
    
    data,diff_data,energy_diff,polar_diff=create_data(mol,basis_list)
    polar_diff.iloc[:,:].plot(marker='o', linestyle='solid')
    plt.axhline(linewidth=2,ls='--',color='k',label="MRA Reference")
    plt.xlabel
    plt.legend(fontsize=12)
    plt.xticks(fontsize=14,rotation = 20)
    plt.title(title,fontsize=20)
    plt.ylabel(yl, fontsize=14)
    save=mol+'-'+basis_list[0]
    if not os.path.exists("figures"):
        os.mkdir("figures")
    if not os.path.exists("tables"):
        os.mkdir("tables")
    save='figures/'+save+'.svg'
    plt.savefig(save)
    
    latex_save=mol+'-'+basis_list[0]
    latex_save='tables/'+latex_save+'.tex'
    data=data.round(decimals=3)
    data.to_latex(latex_save,na_rep=' ')
    
    
    return data


In [4]:
import glob
for g in glob.glob('molecules/*.mol'):
    m=g.split('/')
    mol=m[1].split('.')[0]
    print(mol)


C2H2
He
HF
NH3
Ne
H2
HCl
H2O
C6H6
CH4
BeH2
Be


In [11]:

def get_excited_mol_series(mol,basis):
    d=ExcitedData(mol,'hf')
    g,e=d.compare_dalton(basis)
    molseries=pd.concat([pd.Series(d.converged),e['freq'],e['mad-omega'],e['freq']-e['mad-omega']])
    display(molseries)
    return molseries

In [12]:
def create_excited_comparison_data(basis):
    
    data={}
    for g in glob.glob('molecules/*.mol'):
        m=g.split('/')
        mol=m[1].split('.')[0]
        print(mol)
        if mol!='C6H6':
            data[mol]=get_excited_mol_series(mol,basis)
    excited_data=pd.DataFrame(data).round(3)
    return excited_data

In [18]:
d=create_excited_comparison_data('d-aug-cc-pVQZ')

C2H2
/home/adrianhurtado/testing/madness-test-suite/timings-6-3-22


0    0.000000
0    0.219364
1    0.237923
2    0.237923
3    0.296750
0    0.224847
1    0.237858
2    0.296740
3    0.298625
0   -0.005483
1    0.000065
2   -0.058817
3   -0.001875
dtype: float64

He
/home/adrianhurtado/testing/madness-test-suite/timings-6-3-22


0    1.000000
0    0.776265
1    0.801345
2    0.801345
3    0.801345
0    0.775552
1    0.796940
2    0.796940
3    0.797019
0    0.000713
1    0.004405
2    0.004405
3    0.004326
dtype: float64

HF
/home/adrianhurtado/testing/madness-test-suite/timings-6-3-22


...

0    0.000000
0    0.406295
1    0.406295
2    0.406295
3    0.452574
0    0.406301
1    0.406302
2    0.419700
3    0.452968
0   -0.000006
1   -0.000007
2   -0.013405
3   -0.000394
dtype: float64

BeH2
/home/adrianhurtado/testing/madness-test-suite/timings-6-3-22


0    1.000000
0    0.259482
1    0.259482
2    0.333921
3    0.333921
0    0.259453
1    0.259455
2    0.333737
3    0.333740
0    0.000029
1    0.000027
2    0.000184
3    0.000181
dtype: float64

Be
/home/adrianhurtado/testing/madness-test-suite/timings-6-3-22


0    1.000000
0    0.176939
1    0.176939
2    0.176939
3    0.224906
0    0.176307
1    0.176366
2    0.176392
3    0.247054
0    0.000632
1    0.000573
2    0.000547
3   -0.022148
dtype: float64

In [19]:
d

,C2H2,He,HF,NH3,Ne,H2,HCl,H2O,CH4,BeH2,Be
0,0.000,1.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,1.000
0,0.219,0.776,0.429,0.273,0.674,0.465,0.303,0.318,0.406,0.259,0.177
1,0.238,0.801,0.429,0.326,0.674,0.477,0.303,0.379,0.406,0.259,0.177
2,0.238,0.801,0.535,0.326,0.674,0.481,0.365,0.400,0.406,0.334,0.177
3,0.297,0.801,0.535,0.343,0.739,0.481,0.365,0.410,0.453,0.334,0.225
0,0.225,0.776,0.429,0.326,0.674,0.465,0.302,0.379,0.406,0.259,0.176
1,0.238,0.797,0.429,0.335,0.674,0.477,0.302,0.396,0.406,0.259,0.176
2,0.297,0.797,0.535,0.341,0.674,0.481,0.363,0.401,0.420,0.334,0.176
3,0.299,0.797,0.535,0.367,0.792,0.481,0.375,0.460,0.453,0.334,0.247
0,-0.005,0.001,0.000,-0.053,0.000,0.000,0.001,-0.061,-0.000,0.000,0.001


In [15]:
d=ExcitedData('Be','hf')
d.converged


True

In [16]:
pd.concat([e['freq'],e['mad-omega']])

NameError: name 'e' is not defined

In [ ]:
e